In [1]:
from tkinter import *
from tkinter import messagebox
from tkinter import filedialog as fd
from PIL import ImageTk, Image
import glob
from enum import Enum
import os.path
import threading
import time
import numpy as np
import re
from google.cloud import vision
from google.cloud.vision import types
import cv2
import pandas as pd

In [2]:
class CropImage:
    def __init__(self, x1, y1, x2, y2, category, images):
        self.x1, self.y1, self.x2, self.y2 = (x1, y1, x2, y2)
        self.category = category
        self.images = images
        
    def init_image(self, i):
        self.initial = Image.open(self.images[i])
        path, self.file_name = os.path.split(self.images[i])

    def crop(self):
        if self.category == Category.NAME:
            self.crop_name()
        elif self.category == Category.POWER:
            self.crop_power()
        elif self.category == Category.TOTALKILL:
            self.crop_totalkill()
        elif self.category == Category.KILL:
            self.crop_kill()
        elif self.category == Category.DEAD:
            self.crop_dead()
        elif self.category == Category.SUPPORT:
            self.crop_support()
        
        
    def crop_name(self):
        crop = self.initial.crop((self.x1, self.y1, self.x2, self.y2))
        img = np.array(crop) 
        img = img[:, :, ::-1].copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

        imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
        imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

        imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
        gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

        img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

        ret, img_thresh = cv2.threshold(
            img_blurred, 
            190, #name : 180
            255,
            cv2.THRESH_BINARY
        )
        
        cv2.imwrite('data/croped/name/' + self.file_name, img_thresh)
    
    def crop_power(self):
        crop = self.initial.crop((self.x1, self.y1, self.x2, self.y2))
        img = np.array(crop) 
        img = img[:, :, ::-1].copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

        imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
        imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

        imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
        gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

        img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

        ret, img_thresh = cv2.threshold(
            img_blurred, 
            190, #name : 180
            255,
            cv2.THRESH_BINARY
        )
        
        cv2.imwrite('data/croped/power/' + self.file_name, img_thresh)
    
    def crop_totalkill(self):
        crop = self.initial.crop((self.x1, self.y1, self.x2, self.y2))
        img = np.array(crop) 
        img = img[:, :, ::-1].copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

        imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
        imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

        imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
        gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

        img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

        ret, img_thresh = cv2.threshold(
            img_blurred, 
            190, #name : 180
            255,
            cv2.THRESH_BINARY
        )
        
        cv2.imwrite('data/croped/totalkill/' + self.file_name, img_thresh)
        
    def crop_kill(self):
        crop = self.initial.crop((self.x1, self.y1, self.x2, self.y2))
        
        lower_blue = (0, 0, 0)
        upper_blue =(255, 255, 60)

        img = np.array(crop) 
        img = img[:, :, ::-1].copy() 

        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        img_mask = cv2.inRange(img_hsv, lower_blue, upper_blue)
        img_result = cv2.bitwise_not(img, img)
        img_result1 = cv2.bitwise_and(img_result, img_result, mask=img_mask)
        cv2.imwrite('data/croped/kill/' + self.file_name, img_result1)
        
    def crop_dead(self):
        crop = self.initial.crop((self.x1, self.y1, self.x2, self.y2))
        crop.save('data/croped/dead/' + self.file_name)
        
    def crop_support(self):
        crop = self.initial.crop((self.x1, self.y1, self.x2, self.y2))
        crop.save('data/croped/support/' + self.file_name)        
        

In [3]:
class Category(Enum):
    NAME = 0
    POWER = 1
    TOTALKILL = 2
    KILL = 3
    DEAD = 4
    SUPPORT = 5
    

In [4]:
class Dialog():
    def __init__(self, parent, category, images):
        self.parent = parent
        self.frame = Frame(self.parent)
        self.category = category
        self.images = images
        self.old_x, self.old_y, self.new_x, self.new_y = (0, 0, 0, 0)
        
        self.drag_area = None

        self.img = Image.open(self.images[0])
        w, h = self.img.size
        self.img = self.img.resize((int(w/2), int(h/2)))
        self.image = ImageTk.PhotoImage(self.img)

        w, h = self.img.size
        self.canvas = Canvas(parent, width=w, height=h)
        self.canvas.grid(column=0, row=0)
        self.canvas.create_image(0, 0, anchor=NW, image=self.image)
        self.canvas.image = self.image

        self.parent.bind("<Button-1>", self.motion1)
        self.parent.bind("<ButtonRelease-1>", self.motion2)
        self.parent.bind("<B1-Motion>", self.motion3)

    def motion1(self, event):
        self.canvas.delete(self.drag_area)
        self.drag_area = None
        self.old_x, self.old_y = event.x, event.y

    def motion2(self, event):
        self.new_x, self.new_y = event.x, event.y
        
        msg = messagebox.askokcancel("확인", "제대로 영역지정을 하셨나요?")

        if msg:
            self.parent.destroy()

            if self.category == Category.NAME:
                img = Image.open(self.images[0]).crop((self.old_x*2, self.old_y*2, self.new_x*2, self.new_y*2))

                w, h = img.size
                application.canvas_name = Canvas(application.parent, width=w/2, height=h/2)
                application.canvas_name.grid(column=1, row=2, padx=1, pady=5, columnspan=3)

                img = img.resize((int(w/2), int(h/2)))
                image = ImageTk.PhotoImage(img)
                application.canvas_name.create_image(0, 0, anchor=NW, image=image)
                application.canvas_name.image = image
                
                application.crop_name = CropImage(self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2, Category.NAME, self.images)
                
            elif self.category == Category.POWER:
                img = Image.open(self.images[0]).crop((self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2))

                w, h = img.size
                application.canvas_power = Canvas(application.parent, width=w / 2, height=h / 2)
                application.canvas_power.grid(column=1, row=3, padx=1, pady=5, columnspan=3)

                img = img.resize((int(w / 2), int(h / 2)))
                image = ImageTk.PhotoImage(img)
                application.canvas_power.create_image(0, 0, anchor=NW, image=image)
                application.canvas_power.image = image
                
                application.crop_power = CropImage(self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2, Category.POWER, self.images)
            
            elif self.category == Category.TOTALKILL:
                img = Image.open(self.images[0]).crop((self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2))

                w, h = img.size
                application.canvas_totalkill = Canvas(application.parent, width=w / 2, height=h / 2)
                application.canvas_totalkill.grid(column=1, row=4, padx=1, pady=5, columnspan=3)

                img = img.resize((int(w / 2), int(h / 2)))
                image = ImageTk.PhotoImage(img)
                application.canvas_totalkill.create_image(0, 0, anchor=NW, image=image)
                application.canvas_totalkill.image = image
                
                application.crop_totalkill = CropImage(self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2, Category.TOTALKILL, self.images)
                
            
            elif self.category == Category.KILL:
                img = Image.open(self.images[0]).crop((self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2))

                w, h = img.size
                application.canvas_kill = Canvas(application.parent, width=w / 2, height=h / 2)
                application.canvas_kill.grid(column=1, row=5, padx=1, pady=5, columnspan=3)

                img = img.resize((int(w / 2), int(h / 2)))
                image = ImageTk.PhotoImage(img)
                application.canvas_kill.create_image(0, 0, anchor=NW, image=image)
                application.canvas_kill.image = image
                
                application.crop_kill = CropImage(self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2, Category.KILL, self.images)
                
                
            elif self.category == Category.DEAD:
                img = Image.open(self.images[0]).crop((self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2))

                w, h = img.size
                application.canvas_dead = Canvas(application.parent, width=w / 2, height=h / 2)
                application.canvas_dead.grid(column=1, row=6, padx=1, pady=5, columnspan=3)

                img = img.resize((int(w / 2), int(h / 2)))
                image = ImageTk.PhotoImage(img)
                application.canvas_dead.create_image(0, 0, anchor=NW, image=image)
                application.canvas_dead.image = image
                
                application.crop_dead = CropImage(self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2, Category.DEAD, self.images)
                
            
            elif self.category == Category.SUPPORT:
                img = Image.open(self.images[0]).crop((self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2))

                w, h = img.size
                application.canvas_support = Canvas(application.parent, width=w / 2, height=h / 2)
                application.canvas_support.grid(column=1, row=7, padx=1, pady=5, columnspan=3)

                img = img.resize((int(w / 2), int(h / 2)))
                image = ImageTk.PhotoImage(img)
                application.canvas_support.create_image(0, 0, anchor=NW, image=image)
                application.canvas_support.image = image
                
                application.crop_support = CropImage(self.old_x * 2, self.old_y * 2, self.new_x * 2, self.new_y * 2, Category.SUPPORT, self.images)

    def motion3(self, event):
        if self.drag_area != None:
            self.canvas.delete(self.drag_area)

        w = 2
        self.drag_area = self.canvas.create_rectangle(self.old_x, self.old_y, event.x, event.y, width=w)

In [5]:
class Main(Frame):
    def __init__(self, parent):
        self.parent = parent
        self.frame = Frame(self.parent)
        
        self.name = list()
        self.power = list()
        self.kill = list()
        self.t1 = list()
        self.t2 = list()
        self.t3 = list()
        self.t4 = list()
        self.t5 = list()
        self.dead = list()
        self.support = list()

        self.i = 0
        
        btn_load_img = Button(parent, text='이미지 불러오기', command=self.load_image)
        btn_load_img.grid(column=0, row=0, padx=5, pady=5)
        
        btn_edit_img = Button(parent, text='이미지 편집', command=self.crop_image)
        btn_edit_img.grid(column=1, row=0, padx=5, pady=5)        
        
        btn_detect_img = Button(parent, text="이미지 인식", command=self.detect_image)
        btn_detect_img.grid(column=2, row=0, padx=5, pady=5)
        
        btn_export = Button(parent, text="엑셀파일로", command=self.export_excel)
        btn_export.grid(column=3, row=0, padx=5, pady=5)
        
        self.lbl_status = Label(parent, text='진행 상황')
        self.lbl_status.grid(column=0, row=1, padx=5, pady=5, columnspan=4)
        
        # Name
        btn_get_name = Button(parent, text='이름', command=self.open_nameDialog)
        btn_get_name.grid(column=0, row=2, padx=5, pady=5)
        
        # Power
        btn_get_name = Button(parent, text='투력', command=self.open_powerDialog)
        btn_get_name.grid(column=0, row=3, padx=5, pady=5)
        
        # TotalKill
        btn_get_name = Button(parent, text='총 킬수', command=self.open_totalkillDialog)
        btn_get_name.grid(column=0, row=4, padx=5, pady=5)

        # Kill
        btn_get_kill = Button(parent, text='티어별 킬수', command=self.open_killDialog)
        btn_get_kill.grid(column=0, row=5, padx=5, pady=5)

        # Dead
        btn_get_ds = Button(parent, text='전사', command=self.open_deadDialog)
        btn_get_ds.grid(column=0, row=6, padx=5, pady=5)
        
        # Support
        btn_get_ds = Button(parent, text='원조', command=self.open_supportDialog)
        btn_get_ds.grid(column=0, row=7, padx=5, pady=5)

    def load_image(self):
        self.files = fd.askopenfilenames()
        
        self.load()
           
    def load(self):
        total = len(self.files)
        
        s = '{}/{} 완료'.format(self.i, total)
        self.lbl_status.config(text=s)
        self.parent.update()
        
        fname = os.path.split(self.files[self.i])[1]
        im = Image.open(self.files[self.i])
        im.save('data/original/' + fname)
        
        self.i += 1
        
        s = '{}/{} 완료'.format(self.i, total)
        self.lbl_status.config(text=s)
        self.parent.update()
        
        if self.i == total:
            self.parent.after_cancel(self.job_load) 
            self.i = 0
            self.images = glob.glob('data/original/*')
            messagebox.showinfo("작업 상태", "완료!")
        else:
            self.job_load = self.parent.after(0, self.load)
            
        
    def open_nameDialog(self):
        self.nameDialog = Toplevel(self.parent)
        self.name_app = Dialog(self.nameDialog, Category.NAME, self.images)
        
    def open_powerDialog(self):
        self.powerDialog = Toplevel(self.parent)
        self.power_app = Dialog(self.powerDialog, Category.POWER, self.images)
        
    def open_totalkillDialog(self):
        self.totalkillDialog = Toplevel(self.parent)
        self.totalkill_app = Dialog(self.totalkillDialog, Category.TOTALKILL, self.images)

    def open_killDialog(self):
        self.killDialog = Toplevel(self.parent)
        self.kill_app = Dialog(self.killDialog, Category.KILL, self.images)
        
    def open_deadDialog(self):
        self.deadDialog = Toplevel(self.parent)
        self.dead_app = Dialog(self.deadDialog, Category.DEAD, self.images)
        
    def open_supportDialog(self):
        self.supportDialog = Toplevel(self.parent)
        self.support_app = Dialog(self.supportDialog, Category.SUPPORT, self.images)
        
    def crop_image(self):
        total = len(self.images)
        
        s = '{}/{} 완료'.format(self.i, total)
        self.lbl_status.config(text=s)
        self.parent.update()
        
        self.crop_name.init_image(self.i)
        self.crop_name.crop()
        
        self.crop_power.init_image(self.i)
        self.crop_power.crop()
        
        self.crop_totalkill.init_image(self.i)
        self.crop_totalkill.crop()
        
        self.crop_kill.init_image(self.i)
        self.crop_kill.crop()
        
        self.crop_dead.init_image(self.i)
        self.crop_dead.crop()
        
        self.crop_support.init_image(self.i)
        self.crop_support.crop()
        
        self.paste_image(self.i)
        
        self.i += 1
        
        s = '{}/{} 완료'.format(self.i, total)
        self.lbl_status.config(text=s)
        self.parent.update()
        
        if self.i == total:
            self.parent.after_cancel(self.job_crop) 
            self.i = 0
            messagebox.showinfo("작업 상태", "완료!")
        else:
            self.job_crop = self.parent.after(0, self.crop_image)
 
    def paste_image(self, i):
        path, file_name = os.path.split(self.images[i])
        w, h = (0, 0)

        name = Image.open('data/croped/name/' + file_name)
        if w < name.size[0]:
            w = name.size[0]
        h += name.size[1]

        power = Image.open('data/croped/power/' + file_name)
        if w < power.size[0]:
            w = power.size[0]
        h += power.size[1]

        totalkill = Image.open('data/croped/totalkill/' + file_name)
        if w < totalkill.size[0]:
            w = totalkill.size[0]
        h += totalkill.size[1]

        kill = Image.open('data/croped/kill/' + file_name)
        if w < kill.size[0]:
            w = kill.size[0]
        h += kill.size[1]

        dead = Image.open('data/croped/dead/' + file_name)
        if w < dead.size[0]:
            w = dead.size[0]
        h += dead.size[1]

        support = Image.open('data/croped/support/' + file_name)
        if w < support.size[0]:
            w = support.size[0]
        h += support.size[1]

        new = Image.new("RGB", (w+50, h+50))
        new.paste(name, (0, 0))
        new.paste(power, (0, name.size[1]))
        new.paste(totalkill, (0, name.size[1]+power.size[1]))
        new.paste(kill, (0, name.size[1]+power.size[1]+totalkill.size[1]))
        new.paste(dead, (0, name.size[1]+power.size[1]+totalkill.size[1]+kill.size[1]))
        new.paste(support, (0, name.size[1]+power.size[1]+totalkill.size[1]+kill.size[1]+dead.size[1]))

        new.save('data/pasted/' + file_name)
            
    def detect_image(self):
        total = len(self.images)
        
        s = '{}/{} 완료'.format(self.i, total)
        self.lbl_status.config(text=s)
        self.parent.update()
        
        path, file_name = os.path.split(self.images[self.i])
        texts = self.detect_text('data/pasted/' + file_name)
        tmp = self.change_to_list(texts)
        
        self.name.append(tmp[0])
        self.power.append(self.change_to_number(tmp[1]))
        self.kill.append(self.change_to_number(tmp[2]))
        self.t1.append(self.change_to_number(tmp[3]))
        self.t2.append(self.change_to_number(tmp[4]))
        self.t3.append(self.change_to_number(tmp[5]))
        self.t4.append(self.change_to_number(tmp[6]))
        self.t5.append(self.change_to_number(tmp[7]))
        self.dead.append(self.change_to_number(tmp[8]))
        self.support.append(self.change_to_number(tmp[9]))
        
        
        self.i += 1
        
        s = '{}/{} 완료'.format(self.i, total)
        self.lbl_status.config(text=s)
        self.parent.update()
        
        if self.i == total:
            self.parent.after_cancel(self.job_detect) 
            self.i = 0
            messagebox.showinfo("작업 상태", "완료!")
        else:
            self.job_detect = self.parent.after(0, self.detect_image)
    
    def detect_text(self, path):
        """Detects text in the file."""
        from google.cloud import vision
        import io
        client = vision.ImageAnnotatorClient()

        # [START vision_python_migration_text_detection]
        with io.open(path, 'rb') as image_file:
            content = image_file.read()

        image = vision.types.Image(content=content)

        response = client.text_detection(image=image)
        texts = response.text_annotations

        return texts[0].description

    def change_to_number(self, s_power):
        list_of_numbers = re.findall(r'\d+', s_power)
        return int(''.join(list_of_numbers))
    
    def change_to_list(self, chars):
        tmp_texts = list()
        s = ""
        for c in chars:
            if c != '\n':
                s += c
            else:
                tmp_texts.append(s)
                s = ""

        return tmp_texts
    
            
    def export_excel(self):
        df = pd.DataFrame({'name': self.name,
                            'power':self.power,
                            'all_kill':self.kill,
                            'T1_kill':self.t1,
                            'T2_kill':self.t2,
                            'T3_kill':self.t3,
                            'T4_kill':self.t4,
                            'T5_kill':self.t5,
                            'dead':self.dead,
                            'support':self.support})
    
        filename = fd.asksaveasfilename(filetypes=[('excel file', ',xlsx')], title='Save file as', initialfile='memberinfo.xlsx')
        df.to_excel(filename)
        

In [ ]:
if __name__ == "__main__":
    root = Tk()
    application = Main(root)
    root.resizable(width=False, height=False)
    
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bin/ROKS-78e082836307.json"

    root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\sbyun\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\sbyun\anaconda3\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "<ipython-input-5-bc90ae8e19bf>", line 209, in detect_image
    self.power.append(self.change_to_number(tmp[1]))
  File "<ipython-input-5-bc90ae8e19bf>", line 252, in change_to_number
    return int(''.join(list_of_numbers))
ValueError: invalid literal for int() with base 10: ''
